In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
from sklearn.neural_network import MLPRegressor
import tqdm
import warnings

warnings.filterwarnings("ignore")

random.seed(42)
np.random.seed(42)

## data


def dgp_bivariatenormal(N=10000, cor = 0.9):
    m = lambda x: x[:, 0] ** 2

    x = np.random.multivariate_normal(
        mean=[0, 0], cov=[[1, cor], [cor, 1]], size=N
    )
    data = pd.DataFrame(x, columns=["x1", "x2"])
    data["y"] = m(data[["x1", "x2"]].values) + np.random.normal(0, 0.2, N)
    X = data[["x1", "x2"]]
    y = data["y"]

    return X, y


## pfi


def pfi(model, X, y, feature):
    baseline_score = mean_squared_error(y, model.predict(X))
    X_tilde = X.copy()
    X_tilde[feature] = np.random.permutation(X_tilde[feature])
    score = mean_squared_error(y, model.predict(X_tilde))
    return score - baseline_score


def pfis(model, X, y):
    return {feature: pfi(model, X, y, feature) for feature in X.columns}


def pfiss(model, X_train, y_train, X_test, y_test, repetitions=100):
    scores = pd.DataFrame([], columns=X.columns)
    for ii in tqdm.tqdm(range(repetitions)):
        model.fit(X_train, y_train)
        pfi_scores = pfis(model, X_test, y_test)
        scores = pd.concat([scores, pd.DataFrame(pfi_scores, index=[ii])])
    scores_agg = scores.mean(axis=0).to_frame().T
    scores_agg.index = ["mean"]
    scores_agg.loc["std"] = scores.std(axis=0)
    scores_agg.loc["min"] = scores.min(axis=0)
    scores_agg.loc["max"] = scores.max(axis=0)
    return scores_agg


In [16]:
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
    HistGradientBoostingRegressor,
)
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "MLPRegressor": MLPRegressor(random_state=42, max_iter=1000),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
}
for cor in [0, 0.3, 0.6, 0.9]:
    X, y = dgp_bivariatenormal(N=10000, cor=cor)
    print(f"CORRELATION={cor}")
    print(X.corr())
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    for name, model in models.items():
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_test, model.predict(X_test))
        pfi_scores = pfiss(model, X_train, y_train, X_test, y_test, repetitions=100)
        print(f"{name}: MSE={mse:.4f}, PFI={pfi_scores}")

CORRELATION=0
         x1       x2
x1  1.00000  0.00084
x2  0.00084  1.00000


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 132.42it/s]


LinearRegression: MSE=1.9673, PFI=            x1        x2
mean -0.000013 -0.000476
std   0.000312  0.001135
min  -0.001077 -0.003783
max   0.000583  0.001952


100%|██████████| 100/100 [00:00<00:00, 153.19it/s]


...

MLPRegressor: MSE=0.0413, PFI=            x1        x2
mean  3.502292  0.019697
std   0.086638  0.002050
min   3.266302  0.014664
max   3.695055  0.026765


100%|██████████| 100/100 [05:49<00:00,  3.49s/it]


SVR: MSE=0.0413, PFI=            x1        x2
mean  2.901527  0.209362
std   0.076345  0.012636
min   2.701342  0.185711
max   3.056989  0.246902


100%|██████████| 100/100 [00:02<00:00, 43.45it/s]

KNN: MSE=0.0501, PFI=            x1        x2
mean  2.910489  0.534244
std   0.052216  0.038457
min   2.788326  0.458514
max   3.039082  0.643433


In [17]:
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
    HistGradientBoostingRegressor,
)
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "MLPRegressor": MLPRegressor(random_state=42, max_iter=1000),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
}
for cor in [0, 0.3, 0.6, 0.9]:
    X, y = dgp_bivariatenormal(N=1000, cor=cor)
    print(f"CORRELATION={cor}")
    print(X.corr())
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    for name, model in models.items():
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_test, model.predict(X_test))
        pfi_scores = pfiss(model, X_train, y_train, X_test, y_test, repetitions=100)
        print(f"{name}: MSE={mse:.4f}, PFI={pfi_scores}")

CORRELATION=0
          x1        x2
x1  1.000000  0.031091
x2  0.031091  1.000000


100%|██████████| 100/100 [00:00<00:00, 167.57it/s]


LinearRegression: MSE=1.9893, PFI=            x1        x2
mean -0.014089 -0.015788
std   0.005850  0.006276
min  -0.025946 -0.027154
max  -0.001309 -0.000311


100%|██████████| 100/100 [00:00<00:00, 164.59it/s]


Lasso: MSE=1.9567, PFI=       x1   x2
mean  0.0  0.0
std   0.0  0.0
min   0.0  0.0
max   0.0  0.0


...

MLPRegressor: MSE=0.0618, PFI=            x1        x2
mean  3.111387  0.071770
std   0.224338  0.021022
min   2.367988  0.023501
max   3.624928  0.130512


100%|██████████| 100/100 [00:04<00:00, 20.43it/s]


SVR: MSE=0.0995, PFI=            x1        x2
mean  2.172855  0.273005
std   0.162560  0.069685
min   1.640093  0.110503
max   2.632694  0.427104


100%|██████████| 100/100 [00:00<00:00, 121.05it/s]

KNN: MSE=0.0854, PFI=            x1        x2
mean  2.444552  0.685293
std   0.187168  0.142153
min   1.933683  0.398073
max   2.863029  1.048845
